# RNN Applications
- Time series predication
- Language modeling (Text generation)
- Text sentiment Analysis
- Named entity recongnition
- Translation
- Speech recognition
- Music Composition

In [18]:
import torch
import torch.nn as nn

### Define different cells using nn module

In [15]:
# output is hidden_size
cell = nn.RNN(input_size = 4, hidden_size = 2, batch_first = True)
cell

RNN(4, 2, batch_first=True)

In [19]:
cell = nn.LSTM(input_size = 4, hidden_size = 2, batch_first = True)
cell

LSTM(4, 2, batch_first=True)

In [20]:
cell = nn.GRU(input_size = 4, hidden_size = 2, batch_first = True)
cell

GRU(4, 2, batch_first=True)

In [21]:
# inputs is (batch_size, seq_len, input_size) with batch_first = True 
# hidden is (num_layers, batch_size, hidden_size)

In [29]:
# feed letters as inputs
# One hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [31]:
# batch_size, seq_len, input_size
inputs = torch.autograd.Variable(torch.Tensor([[h]]))
inputs.shape

torch.Size([1, 1, 4])

In [36]:
# num_layers, batch_size, hidden_size (Initialize randome)
hidden = torch.autograd.Variable(torch.randn(1, 1, 2))
hidden.shape

torch.Size([1, 1, 2])

In [40]:
out , hidden = cell(inputs, hidden)
# Expact two values as output 
print(out) 
print(hidden)

tensor([[[0.1223, 0.0739]]], grad_fn=<TransposeBackward0>)
tensor([[[0.1223, 0.0739]]], grad_fn=<ViewBackward>)


### Build RNN for multiple inputs

In [43]:
# batch_size, seq_len, input_size
cell = nn.RNN(input_size = 4, hidden_size = 2, batch_first = True)
inputs = torch.autograd.Variable(torch.Tensor([[h, e, l, l, o]]))
inputs.shape

torch.Size([1, 5, 4])

In [44]:
# num_layers, batch_size, hidden_size (Initialize randome)
hidden = torch.autograd.Variable(torch.randn(1, 1, 2))
out , hidden = cell(inputs, hidden)
out

tensor([[[-0.9180,  0.6105],
         [-0.9332, -0.1493],
         [-0.8318, -0.4902],
         [-0.7867, -0.3557],
         [-0.9283,  0.5129]]], grad_fn=<TransposeBackward0>)

### Build Multiple Batch input

In [49]:
# batch_size, seq_len, input_size
cell = nn.RNN(input_size = 4, hidden_size = 2, batch_first = True)
inputs = torch.autograd.Variable(torch.Tensor([[h, e, l, l, o],
                                               [e, o, l, l, l],
                                               [l, l, e, e, l]]))
# Notice batch_first = True
# batch_size = 3
# seq_len = 5
# input_size = 4
inputs.shape

torch.Size([3, 5, 4])

In [50]:
# num_layers, batch_size, hidden_size (Initialize randome)
hidden = torch.autograd.Variable(torch.randn(1, 3, 2))
out , hidden = cell(inputs, hidden)
out

tensor([[[-0.3909, -0.8278],
         [ 0.2668, -0.8183],
         [ 0.7557, -0.8867],
         [ 0.7022, -0.8637],
         [ 0.2434, -0.5760]],

        [[-0.6912, -0.9124],
         [ 0.5885, -0.7042],
         [ 0.6883, -0.8813],
         [ 0.7109, -0.8668],
         [ 0.7046, -0.8668]],

        [[ 0.8152, -0.9416],
         [ 0.7049, -0.8575],
         [-0.0342, -0.7453],
         [ 0.1342, -0.8053],
         [ 0.7696, -0.8920]]], grad_fn=<TransposeBackward0>)

### Train the network

We want to predict the following sequence:
- h --> i
- i --> h
- h --> e
- e --> l
- l --> l
- l --> o

There are five **input**  letters: h i e l  o 

There are fine **output** letters: h i e l  o 

Design our Loss function using **cross entropy**


In [56]:
index2char = ['h', 'i', 'e', 'l', 'o']
h = [1, 0, 0, 0, 0]
i = [0, 1, 0, 0, 0]
e = [0, 0, 1, 0, 0]
l = [0, 0, 0, 1, 0]
o = [0, 0, 0, 0, 1]
# teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]] #hihell
x_one_hot = [[h, i, h, e, l, l]]
y_data = [1, 0, 2, 3, 3, 4]

In [57]:
cell = nn.RNN(input_size = 5, hidden_size = 5, batch_first = True)
inputs = torch.autograd.Variable(torch.Tensor(x_one_hot))
# Notice batch_first = True
# batch_size = 1
# seq_len = 6
# input_size = 5
inputs.shape

torch.Size([1, 6, 5])

In [60]:
# num_layers, batch_size, hidden_size (Initialize randome)
hidden = torch.autograd.Variable(torch.LongTensor(y_data))
hidden.shape

torch.Size([6])

In [63]:
num_classes = 5
input_size = 5
hidden_size = 5
batch_size = 1
sequence_length = 1
num_layers = 1

In [64]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)

    def forward(self, x, hidden):
        x = x.view(batch_size, sequence_length, input_size)
        out , hidden = self.rnncell(inputs, hidden)
        # output should be 5 (num_classes)
        out = out.view(-1, num_classes)
        return hidden, out
    
    def init_hidden(self):
        return torch.autograd.Variable(torch.zeros(num_layers, batch_size, hidden_size))